# Basic RAG - From Scratch!!

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application.

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

###  Activity #1: First let's explore embeddings and cosine similarity

In [23]:
import numpy as np
from numpy.linalg import norm

def cosine_similarity(vec_1, vec_2):
  return np.dot(vec_1, vec_2) / (norm(vec_1) * norm(vec_2))

Now let's use the `text-embedding-3-small` embedding model (more on that in a second) to embed two sentences. In order to use this embedding model endpoint - we'll need to provide our OpenAI API key!

In [11]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

Then, define the embedding model. Credit goes to AI Makerspace for the library.

In [21]:
from psi.openai_utils.embedding import EmbeddingModel

embedding_model = EmbeddingModel()

Now, let's explore cosine similiarity of 2 embeddings.

In [24]:
puppy_sentence = "I love puppies!"
dog_sentence = "I hate LLMs!"
puppy_vector = embedding_model.get_embedding(puppy_sentence)
dog_vector = embedding_model.get_embedding(dog_sentence)
cosine_similarity(puppy_vector, dog_vector)

np.float64(0.3407870581002385)

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### 🎯 Breakout Room - Group Discussion: 

Explore how cosine similarity works and discuss your observations with the group. Write down final answer below.
</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Observations :

* Cosine similarity works as a metric used to measure how similar two vectors are, regardless of their magnitude. In the context of RAG (Retrieval Augmented Generation), cosine similarity is being utilized to compare text(embeddings), find relevant context given a chunk of documents, given a user query, and capture semantic understanding through cosine similarity. We can observe that we used cosine similarity to compare the embeddings of similar sentences, even though they use different words, but express the same sentiments, resulting in a high cosine similarity.
</span>



## Let's RAG-ing Time!

- Task 1: Imports and Utilities
- Task 2: Documents
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation
  - 🚧 Activity #1: Augment RAG

Here's a more visual representation of the task to be performed.

![RAG Steps](images/ragging.png)

## Import Wall

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [2]:
from psi.text_utils import TextFileLoader, CharacterTextSplitter
from psi.vectordatabase import VectorDatabase
import asyncio

#  need  this support async in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

### Step 1: Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.

> NOTE: We're using text from Noli Me Tangere as our sample data. This data is largely irrelevant as we want to focus on the mechanisms of RAG, which includes out data's shape and quality - but not specifically what the contents of the data are. 


In [6]:
text_loader = TextFileLoader("data/noli.txt")
documents = text_loader.load_documents()
len(documents)

1

In [7]:
def print_document_portion(doc, start, end, title="Document Portion"):
    print(f"\n{'='*50}")
    print(f"{title} (characters {start}-{end})")
    print(f"{'='*50}")
    print(doc[start:end])

# Usage
print_document_portion(documents[0], 0, 100, "Beginning")
print_document_portion(documents[0], 2000, 2500, "Middle Section")



Beginning (characters 0-100)
The Project Gutenberg eBook of The Social Cancer: A Complete English Version of Noli Me Tangere
    

Middle Section (characters 2000-2500)
orical sketches. The reader
    flies in his express train in a few minutes through a couple
    of centuries. The centuries pass more slowly to those to
    whom the years are doled out day by day. Institutions grow
    and beneficently develop themselves, making their way into
    the hearts of generations which are shorter-lived than they,
    attracting love and respect, and winning loyal obedience;
    and then as gradually forfeiting by their shortcomings
    the allegiance which had been 


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [8]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

1283

Let's take a look at some of the documents we've managed to split.

In [9]:
split_documents[0:1]

['The Project Gutenberg eBook of The Social Cancer: A Complete English Version of Noli Me Tangere\n    \nThis ebook is for the use of anyone anywhere in the United States and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever. You may copy it, give it away or re-use it under the terms\nof the Project Gutenberg License included with this ebook or online\nat www.gutenberg.org. If you are not located in the United States,\nyou will have to check the laws of the country where you are located\nbefore using this eBook.\n\nTitle: The Social Cancer: A Complete English Version of Noli Me Tangere\n\nAuthor: José Rizal\n\nTranslator: Charles E. Derbyshire\n\nRelease date: October 1, 2004 [eBook #6737]\n                Most recently updated: April 13, 2010\n\nLanguage: English\n\nCredits: Produced by Jeroen Hellingman.\n\n\n*** START OF THE PROJECT GUTENBERG EBOOK THE SOCIAL CANCER: A COMPLETE ENGLISH VERSION OF NOLI ME TANGERE ***\n\n\n\n\nProduced by Jeroen Hel

## Step 2: Compute Embeddings per Chunk

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?
2. What technique does OpenAI use to achieve this?

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer :

* Answer to Question #1: To modify the embedding dimension, we can specify the dimension by passing it in the dimensions parameter of the API.
* Answer to Question #2: To achieve this, OpenAI utilizes Matryoshka Representation Learning (MRL) that enables a model to output representations of varying sizes.
</span>


> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1, and [this documentation](https://platform.openai.com/docs/guides/embeddings/use-cases) for an answer to question #2!


We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [ ]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### ❓ Assignment Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

</div>

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:

* The benefits of using an `async` approach to collecting embeddings include concurrency and improved performance, allowing for multiple API requests to be made simultaneously, significantly reducing the total processing time when handling a large volume of documents. Additionally, it enables non-blocking operations, meaning the program doesn't freeze and remains responsive while waiting for API responses; other tasks can continue running as embedding requests are processed. Another benefit is scalability; it can handle hundreds or thousands of documents more efficiently.

</div>

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

## Step 3: Finding Best Match with Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [14]:
vector_db.search_by_text("Who is Capitan Tiago?", k=5)

[('f Capitan\nTiago with a telegram, to open which he naturally requested the\npermission of the others, who very naturally begged him to do so. The\nworthy capitan at first knitted his eyebrows, then raised them;\nhis face became pale, then lighted up as he hastily folded the paper\nand arose.\n\n"Gentlemen," he announced in confusion, "his Excellency the\nCaptain-General is coming this evening to honor my house." Thereupon he\nset off at a run, hatless, taking with him the message and his napkin.\n\nHe was followed by exclamations and questions, for a cry of\n"Tulisanes!" would not have produced greater effect. "But,\nlisten!" "When is he coming?" "Tell us about it!" "His Excellency!" But\nCapitan Tiago was already far away.\n\n"His Excellency is coming and will stay at Capitan Tiago\'s!" exclaimed\nsome without taking into consideration the fact that his daughter\nand future son-in-law were present.\n\n"The choice couldn\'t be better," answered the latter.\n\nThe friars gazed at one

## Task 5: Generate the Response

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4o-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

</div>


<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:

* To achieve more reproducible outputs across API calls, we can set the seed parameter to any integer of our choice and use the same value across requests we'd like to have deterministic outputs for. Additionally, ensure all other parameters, such as prompt or temperature, remain the same, and set top_p to a very low number. By following these steps, we can obtain reproducible results.

</div>


> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

### Creating and Prompting OpenAI's `gpt-4o-mini`!

Let's tie all these together and use it to prompt `gpt-4o-mini`!

In [15]:
from psi.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from psi.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages)

In [16]:
print(response)

Writing a loop in Python can vary based on the context and the specific task you want to achieve. Here are a few common ways to write loops, along with some tips to make them effective and readable.

### 1. `for` Loop
A `for` loop is commonly used for iterating over a sequence (like a list, tuple, or string).

```python
# Example: Looping through a list
fruits = ['apple', 'banana', 'cherry']
for fruit in fruits:
    print(fruit)
```

### 2. `while` Loop
A `while` loop continues until a certain condition is false.

```python
# Example: Using a while loop
count = 0
while count < 3:
    print("Count is:", count)
    count += 1
```

### 3. Using `enumerate()`
When you need both the index and the value from a list, `enumerate()` can be very handy.

```python
# Example: Looping with index
fruits = ['apple', 'banana', 'cherry']
for index, fruit in enumerate(fruits):
    print(index, fruit)
```

### 4. List Comprehensions
If you're looking to create a new list based on an existing one, conside

## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [17]:
RAG_SYSTEM_TEMPLATE = """You are a knowledgeable assistant that answers questions based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't know"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "concise",
        "response_length": "brief"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

Now we can create our pipeline!

In [18]:
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

In [19]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    "Who is Capitan Tiago?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")


Response: Capitan Tiago is a character described as low in stature, with a clear complexion, a corpulent figure, and a full face, which some attribute to a "liberal supply of fat," seen by his admirers as a gift from Heaven and by his enemies as the result of exploiting the poor (Source 2). He is depicted as a somewhat cautious and prudent man, particularly in the context of social interactions and decisions he must make concerning his daughter, Maria Clara. Capitan Tiago has respect for a Chinese man who claims to be a prophet and a physician, having predicted the birth of Maria Clara (Source 3).

He is also noted for hosting an important visitor, the Captain-General, who is expected to arrive at his house, an announcement that causes excitement among others present (Source 1). Overall, he is a significant figure in his community, involved in social and governmental affairs highlighted by the presence of guests from the government (Source 3).

Context Count: 3
Similarity Scores: ['Sou

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### 🎯 Breakout Room - Group Discussion: 

Explore different prompts and check if RAG is able to correctly answer the question. Write down final answer below.
</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Observations: 

* The RAG system follows the constraints, citing its context and responding with "I don't know" when the answer is not found in the retrieved chunks. Furthermore, the RAG system adheres to its established RAG system and user prompt template, including its behavior and response structure. The RAG system successfully answered the specified question.
</div>